In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import datasets, linear_model,preprocessing
from datetime import datetime
import gc
%matplotlib inline
from IPython.display import display, HTML
from pprint import pprint
import time
import nltk.corpus
from nltk.stem.snowball import SnowballStemmer
from ml_metrics import rmse,rmsle

In [2]:
#take 1 CSV, then split it to 3..
class FeatureEngineering:

    def __init__(self, ValidationStart, ValidationEnd, trainHdfPath, trainHdfFile, testHdfPath1, testHdfPath2, testHdfFile, 
                 testTypes, trainTypes, trainCsvPath, testCsvPath, maxLag=0):
        self.ValidationStart = ValidationStart
        self.ValidationEnd = ValidationEnd
        self.maxLag = maxLag
        self.trainHdfPath = trainHdfPath
        self.trainHdfFile = trainHdfFile
        self.testHdfPath1 = testHdfPath1
        self.testHdfPath2 = testHdfPath2
        self.testHdfFile = testHdfFile
        self.testTypes = testTypes
        self.trainTypes = trainTypes
        self.trainCsvPath = trainCsvPath
        self.testCsvPath = testCsvPath
        
    @staticmethod
    def __printDataFrameBasics__(data):
        display(data.head(2))
        #print data.dtypes
        gc.collect()
        print(data.info(memory_usage=True))
        
    @staticmethod    
    def changeIndexTypeToLowerMemory(data):
        ##########
        #This is very critical, i accept max number is 2^32. Also, if don't do that, memory gets so much higher..
        ##########
        #data.reset_index(inplace=True)
        #data.drop("index",axis=1, inplace=True)
        #data.index = data.index.astype('uint32')
        gc.collect()
        
    def ReadHdf(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''            
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_hdf(self.trainHdfPath,self.trainHdfFile)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
            
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1 = pd.read_hdf(self.testHdfPath1,self.testHdfFile)
            self.test2 = pd.read_hdf(self.testHdfPath2,self.testHdfFile)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
        
    def ReadCsv(self, trainOrTestOrBoth):
        '''Reads and holds Df in memory'''
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth == 'both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            tempTest = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes)
            self.test1 = tempTest.loc[tempTest.Semana.values == self.ValidationStart]
            self.test2 = tempTest.loc[tempTest.Semana.values == self.ValidationEnd]
            del tempTest
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
            
    @staticmethod
    def ConvertCsvToHdf(csvPath, HdfPath, HdfName, ColumnTypeDict ):
        tempDf = pd.read_csv(csvPath, usecols=ColumnTypeDict.keys(), dtype=ColumnTypeDict,index=False)
        tempDf.to_hdf(HdfPath, HdfName, format='t')
        del tempDf
        gc.collect()
        print "ConvertCsvToHdf is done.."

    def Preprocess(self, trainOrTestOrBoth, columnFunctionTypeList):
        '''columnFunctionTypeList = [ ['C1',Func1,Type], ['C2',Func2,Type],..    ]'''
        for column, func, localType in columnFunctionTypeList:
            if trainOrTestOrBoth == 'train' or trainOrTestOrBoth =='both':
                self.train.loc[:,column] =  np.apply_along_axis(func,0,FE.train[column].values).astype(localType)
                #np.apply_along_axis(lambda x: x+1,0,FE.train["Semana"]).astype("int32")
            if trainOrTestOrBoth == 'test' or trainOrTestOrBoth == 'both':
                self.test1.loc[:,column] =  np.apply_along_axis(func,0,FE.test1[column].values).astype(localType)
                self.test2.loc[:,column] =  np.apply_along_axis(func,0,FE.test2[column].values).astype(localType)
        gc.collect()
        
    def SaveDataFrameToHdf(self,trainOrTestOrBoth):
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.to_hdf(self.trainHdfPath, self.trainHdfFile, format='t', index="False")
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1.to_hdf(self.testHdfPath1, self.testHdfFile, format='t', index="False")
            self.test2.to_hdf(self.testHdfPath2, self.testHdfFile, format='t', index="False")
        
    def AddDemandaGeneralMean(self,trainOrTestOrBoth): 
        #self.train.loc[:,"DemandaGeneralMean"] = self.train["Demanda_uni_equil"].loc[
         #   self.train.loc[:,'Semana'] < 10].mean().astype("float32")
            
        meanOfDemanda = self.train["Demanda_uni_equil"].values.mean().astype("float32")
        
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.loc[:,"DemandaGeneralMean"] = meanOfDemanda
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test1.loc[:,"DemandaGeneralMean"] = meanOfDemanda
            self.test2.loc[:,"DemandaGeneralMean"] = meanOfDemanda
        
        #self.train.loc[:,"DemandaGeneralMean"] = self.train["Demanda_uni_equil"].values[
        #(self.train.loc[:,'Semana'].values < self.ValidationStart).values].mean().astype("float32")
        gc.collect()
        
    '''ConfigElements(0,[ ("A",["Semana","Agencia_ID"],["count","count"]),'''
    def AddConfigurableFeaturesToTrain(self, config):
        if config.lag > self.maxLag:
            self.maxLag = config.lag
        
        tempData = self.train[self.train['Semana'].values <= (self.ValidationEnd - config.lag)]
        #display(tempData)
        if(config.lag != 0):
            tempData.loc[:,'Semana'] = tempData['Semana'].values + config.lag
        #display(tempData)
        
        #Means iterative.. eliminate as long as np.nan exists..If there is already one, don't create, use the existing
        if config.targetVariable != "" and  config.targetVariable not in self.train.columns:
            self.train.loc[:,config.targetVariable] = np.nan
            self.test1.loc[:,config.targetVariable] = np.nan
            
            if config.lag != 1:
                self.test2.loc[:,config.targetVariable] = np.nan
        
        for name,groups,aggregate in config.nameAndGroups:
            if name not in self.train.columns:
                print "{} is not in columns..".format(name)            
                
                groupedDataframe = tempData[groups+['Demanda_uni_equil']].copy().groupby(groups).agg(aggregate[0])
                gc.collect()
                #groupedDataframe.columns = groupedDataframe.columns.droplevel(0)
                groupedDataframe.columns = [name]
                
                #This is means of the counts of the semana-columns tuples!..!!!
                #If no lag and mean, mean of the columns without semana!!..
                #If there is lag and count, count of the columns x weeks before
                #If there is lag and mean, mean of the columns x weeks before
                #if(config.lag == 0 and aggregate == "count"):
                if(len(aggregate)>1):
                    groupedDataframe.reset_index(inplace=True)
                    groupedDataframe.drop("Semana",axis=1, inplace=True)
                    groups = groups[1:]
                    groupedDataframe = groupedDataframe.groupby(groups).agg(aggregate[1])
                    groupedDataframe.columns = [name]
                    gc.collect()
                
                display(groupedDataframe.head(2))
                self.train = self.train.merge( groupedDataframe, left_on=groups,
                    right_index=True, how='left', sort=False,copy=False)
                gc.collect()
                self.test1 = self.test1.merge( groupedDataframe, left_on=groups,
                    right_index=True, how='left', sort=False,copy=False)
                gc.collect()
                if config.lag != 1:
                    self.test2 = self.test2.merge( groupedDataframe, left_on=groups,
                        right_index=True, how='left', sort=False,copy=False)
                
                del groupedDataframe
                gc.collect()
            else:
                print "{} is in columns..".format(name)
            
            display(self.train.head(2))
            display(self.test1.head(2))
            display(self.test2.head(2))
            
            #Means iterative..!!!!!
            if config.targetVariable != "":
                self.train.loc[pd.isnull(self.train[config.targetVariable].values), 
                    config.targetVariable] = self.train.loc[pd.isnull(self.train[config.targetVariable].values)
                    , name].values
                self.test1.loc[pd.isnull(self.test1[config.targetVariable].values), 
                    config.targetVariable] = self.test1.loc[pd.isnull(self.test1[config.targetVariable].values),
                    name].values
                if config.lag != 1:
                    self.test2.loc[pd.isnull(self.test2[config.targetVariable].values), 
                        config.targetVariable] = self.test2.loc[pd.isnull(self.test2[config.targetVariable].values)
                        , name].values
                    
                count = self.test1[config.targetVariable].isnull().sum()
                print "Count of missing numbers after {} in validation part 1 in column {} is {}".format(name, 
                    config.targetVariable,str(count))
                if config.lag != 1:
                    count = self.test2.loc[:,config.targetVariable].isnull().sum()
                    print "Count of missing numbers after {} in validation part 2 in column {} is {}".format(name, 
                        config.targetVariable,str(count))
                
                
                #display(self.train)
                #If column is already in Dataframe and we want to fill target variable, this deletes columns!!!
                if(config.deleteColumns):
                    self.train.drop(name, axis=1, inplace=True)
                    self.test1.drop(name, axis=1, inplace=True)
                    if config.lag != 1:
                        self.test2.drop(name, axis=1, inplace=True)
                gc.collect()
                #Only in tesst
                #if count == 0:
                 #   break
        del tempData
        display(self.train.head(2))
        display(self.test1.head(2))
        display(self.test2.head(2))
        gc.collect()
        return 
    
    def DeleteLaggedWeeksFromTrain(self):
        self.train = self.train[self.train['Semana'].values >= (3 + self.maxLag)]
        gc.collect()
        display(self.train.head(2))
        
    def ReadFirstNRowsOfACsv(self, nrows, trainOrTestOrBoth) :
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes, nrows = nrows)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            tempTest = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes, nrows = nrows*2)
            self.test1 = tempTest.loc[tempTest.Semana == self.ValidationStart]
            self.test2 = tempTest.loc[tempTest.Semana == self.ValidationEnd]
            del tempTest
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test1)
            FeatureEngineering.changeIndexTypeToLowerMemory(self.test2)
            FeatureEngineering.__printDataFrameBasics__(self.test1)
            FeatureEngineering.__printDataFrameBasics__(self.test2)
    
    #Use when concatanating train and validation before predict test for example..
    def AppendTestToTrain(self,deleteTest = True):
        self.train = self.train.append(self.test1,ignore_index=True)
        gc.collect()
        if(deleteTest):
            del self.test1
            gc.collect()
        try:
            self.train = self.train.append(self.test2,ignore_index=True)
            gc.collect()
            if(deleteTest):
                del self.test2
                gc.collect()
        except:
            pass
        #BAD PERFORMANCE!!
    #Split train data to train and test1 and test2 (validation)
    #def SplitTrainToTestUsingValidationStart(self):
     #   boolCondition = self.train.Semana == self.ValidationStart
      #  self.test1 = self.train.loc[boolCondition]
       # self.train.drop((self.train.loc[boolCondition].index), axis=0,inplace=True)
        
       # boolCondition = self.train.Semana == self.ValidationEnd
       # self.test2 = self.train.loc[boolCondition]
       # self.train.drop((self.train.loc[boolCondition].index), axis=0,inplace=True)
      #  del boolCondition
      #  gc.collect()
    
    #Reaches 3x memory from train, because of test1, test2 and train itself at the end.. GC fixed in the end..
    def SplitTrainToTestUsingValidationStart(self):
        boolCondition = self.train.Semana.values == self.ValidationStart
        self.test1 = self.train[boolCondition]
        boolCondition = self.train.Semana.values == self.ValidationEnd
        self.test2 = self.train[boolCondition]
        FE.train = FE.train[ FE.train.Semana.values < FE.ValidationStart ]
        del boolCondition
        gc.collect()

In [3]:
class ConfigElements:
    def __init__(self, lag, nameAndGroups, targetVariable="", deleteColumns = False):
        self.lag = lag
        self.nameAndGroups = nameAndGroups
        #If there is target variable, then 5 4 3 2 1, fill the np.nans..Else hold them all in Dataframe..
        self.targetVariable = targetVariable
        self.deleteColumns = deleteColumns

In [4]:
parameterDict =       {"ValidationStart":10, 
 "ValidationEnd":11,
   "maxLag":3,
    "trainHdfPath":'../../input/train_full_wz2.h5',
    "trainHdfFile":"train",
    "testHdfPath1":"../../input/test1_full_wz2.h5",
    "testHdfPath2":"../../input/test2_full_wz2.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Venta_uni_hoy':np.uint16, 'Venta_hoy':np.float32,
                    'Dev_uni_proxima': np.uint32, 'Dev_proxima':np.float32,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train.csv'   ,
    "testCsvPath":'../../input/test.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

{'trainCsvPath': '../../input/train.csv', 'maxLag': 3, 'testTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>, 'id': <type 'numpy.uint32'>}, 'testHdfFile': 'test', 'trainTypes': {'Dev_proxima': <type 'numpy.float32'>, 'Venta_uni_hoy': <type 'numpy.uint16'>, 'Cliente_ID': <type 'numpy.uint32'>, 'Demanda_uni_equil': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Venta_hoy': <type 'numpy.float32'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Dev_uni_proxima': <type 'numpy.uint32'>, 'Semana': <type 'numpy.uint8'>}, 'testHdfPath1': '../../input/test1_full_wz2.h5', 'ValidationEnd': 11, 'testHdfPath2': '../../input/test2_full_wz2.h5', 'testCsvPath': '../../input/test.csv', 'ValidationStart': 10, 'trainHdfFile': 'train', 'trainHdfPath': '../../input

In [5]:
#FE.ReadCsv('both')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74180464 entries, 0 to 74180463
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    uint32
dtypes: float32(2), uint16(4), uint32(3), uint8(2)
memory usage: 2.1 GB
None


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
2,2,10,2045,1,2831,4549769,32940
7,7,10,1612,1,2837,4414012,35305


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3538385 entries, 2 to 6999247
Data columns (total 7 columns):
id             uint32
Semana         uint8
Agencia_ID     uint16
Canal_ID       uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtypes: uint16(3), uint32(2), uint8(2)
memory usage: 81.0 MB
None


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3460866 entries, 0 to 6999250
Data columns (total 7 columns):
id             uint32
Semana         uint8
Agencia_ID     uint16
Canal_ID       uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtypes: uint16(3), uint32(2), uint8(2)
memory usage: 79.2 MB
None


In [6]:
FE.test2[(FE.test2.id == 0)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305


## Merge town and products..

In [7]:
townstate = pd.read_csv("../../input/town_state.csv", encoding='utf-8')
townstate['Town_ID']=townstate['Town'].str[:4]
states = townstate['State']
le = preprocessing.LabelEncoder()
townstate['State_ID']=le.fit_transform(states)
townstate = townstate.drop(['Town', 'State'], axis=1)
townstate = townstate.astype('uint16')
townstate[['State_ID']] =townstate[['State_ID']] .astype('uint8')
FE.train = pd.merge(FE.train, townstate, on='Agencia_ID', how='left', sort=False,copy=False)
gc.collect()
FE.test1 = pd.merge(FE.test1, townstate, on='Agencia_ID', how='left', sort=False,copy=False)
gc.collect()
FE.test2 = pd.merge(FE.test2, townstate, on='Agencia_ID', how='left', sort=False,copy=False)
gc.collect()

84

In [8]:
products = pd.read_csv("../../input/producto_tabla.csv")
products['short_name'] = products.NombreProducto.str.extract('^(\D*)', expand=False)
products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
w = products.NombreProducto.str.extract('(\d+)(Kg|g)', expand=True)
products['weight'] = w[0].astype('float') * w[1].map({'Kg': 1000, 'g': 1})
products['pieces'] = products.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')

products['short_name_processed'] = (products['short_name'].
                                    map(lambda x: " ".
                                        join([i for i in x.lower().split() 
                                              if i not in nltk.corpus.stopwords.words("spanish")])))
stemmer = SnowballStemmer("spanish")
products['short_name_processed'] = (products['short_name_processed'].
                                    map(lambda x: " ".join([stemmer.stem(i) for i in x.lower().split()])))

le = preprocessing.LabelEncoder()

products['Prod_name_ID']=le.fit_transform(products['short_name_processed'])
products['Brand_ID']=le.fit_transform(products['brand'])

products = products.drop(['short_name', 'brand', 'short_name_processed', 'NombreProducto'], axis=1)
products.fillna(value=0, inplace=True)
products[['pieces','Brand_ID']] = products[['pieces','Brand_ID']].astype('uint8')
products[['Producto_ID','weight', 'Prod_name_ID']] = products[['Producto_ID','weight', 'Prod_name_ID']].astype('uint16')
FE.train = pd.merge(FE.train, products, on='Producto_ID', how='left', sort=False,copy=False)
gc.collect()
FE.test1 = pd.merge(FE.test1, products, on='Producto_ID', how='left', sort=False,copy=False)
gc.collect()
FE.test2 = pd.merge(FE.test2, products, on='Producto_ID', how='left', sort=False,copy=False)
gc.collect()

14

In [9]:
FE.test2[(FE.test2.id == 0)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23


## Check NA values here.

In [10]:
FE.train[FE.train.isnull().any(axis=1)]

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID


In [11]:
FE.test1[FE.test1.isnull().any(axis=1)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID


In [12]:
FE.test2[FE.test2.isnull().any(axis=1)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID


## Last State of Train, test1 and test2..

In [13]:
display(FE.train.head(2))
display(FE.test1.head(2))
display(FE.test2.head(2))

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3,2008,16,120,2,709,4
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4,2008,16,135,2,712,4


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4


## Convert to Log..

In [14]:
FE.train.loc[:,"Demanda_uni_equil"] = np.log1p (FE.train["Demanda_uni_equil"].values)
FE.train.loc[:,"Demanda_uni_equil_original"] =  np.round( np.expm1(FE.train["Demanda_uni_equil"]))
gc.collect()

649

## Split Train to test1 test2

In [15]:
#FE.SplitTrainToTestUsingValidationStart()

## Add DemandaNotEqualTheDifferenceOfVentaUniAndDev to model products delivered later..

In [16]:
FE.train.loc[:,"DemandaNotEqualTheDifferenceOfVentaUniAndDev"] = FE.train.Demanda_uni_equil_original.values < (
    FE.train.Venta_uni_hoy.values - FE.train.Dev_uni_proxima.values)
gc.collect()

7

## Delete Demanda = 0  and Venta = 0

In [17]:
#FE.train = FE.train[(FE.train.Demanda_uni_equil.values != 0) & (FE.train.Venta_uni_hoy.values != 0)]
#gc.collect()

In [18]:
FE.SaveDataFrameToHdf('both')

In [5]:
FE.ReadHdf('both')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,1.386294,2008,16,120,2,709,4,3.0,False,1.418015
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,1.609438,2008,16,135,2,712,4,4.0,False,1.279644


<class 'pandas.core.frame.DataFrame'>
Int64Index: 74180464 entries, 0 to 74180463
Data columns (total 20 columns):
Semana                                          uint8
Agencia_ID                                      uint16
Canal_ID                                        uint8
Ruta_SAK                                        uint16
Cliente_ID                                      uint32
Producto_ID                                     uint16
Venta_uni_hoy                                   uint16
Venta_hoy                                       float32
Dev_uni_proxima                                 uint32
Dev_proxima                                     float32
Demanda_uni_equil                               float64
Town_ID                                         uint16
State_ID                                        uint8
weight                                          uint16
pieces                                          uint8
Prod_name_ID                                    uint16
Brand_

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23,1.098612
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23,1.892663


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3538385 entries, 0 to 3538384
Data columns (total 14 columns):
id              uint32
Semana          uint8
Agencia_ID      uint16
Canal_ID        uint8
Ruta_SAK        uint16
Cliente_ID      uint32
Producto_ID     uint16
Town_ID         uint16
State_ID        uint8
weight          uint16
pieces          uint8
Prod_name_ID    uint16
Brand_ID        uint8
Lag0            float64
dtypes: float64(1), uint16(6), uint32(2), uint8(5)
memory usage: 138.4 MB
None


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.006477


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3460866 entries, 0 to 3460865
Data columns (total 14 columns):
id              uint32
Semana          uint8
Agencia_ID      uint16
Canal_ID        uint8
Ruta_SAK        uint16
Cliente_ID      uint32
Producto_ID     uint16
Town_ID         uint16
State_ID        uint8
weight          uint16
pieces          uint8
Prod_name_ID    uint16
Brand_ID        uint8
Lag0            float64
dtypes: float64(1), uint16(6), uint32(2), uint8(5)
memory usage: 135.3 MB
None


In [6]:
FE.test2[(FE.test2.id == 0)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438


## Grouping for Lag0

In [21]:
#FE.train = FE.train.drop(['Lag0'], axis=1)  ## run these again, then comment out.

In [22]:
configLag0Target1DeleteColumnsFalse = ConfigElements(0,[ ("SPClRACh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClRA0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPClRCh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClACh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClR0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPClA0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPClCh0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPCl0_mean",[
                                                              "Producto_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPR0_mean",[
                                                              "Producto_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPA0_mean",[
                                                              "Producto_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPCh0_mean",[
                                                              "Producto_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPT0_mean",[
                                                              "Producto_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPSt0_mean",[
                                                              "Producto_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SP0_mean",[
                                                              "Producto_ID"],
                                                          ["mean"]),
                                                      ("SPnClRACh0_mean",[  ##PRODNAMES START
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClRA0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPnClRCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClACh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClR0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPnClA0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnClCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnCl0_mean",[
                                                              "Prod_name_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPnR0_mean",[
                                                              "Prod_name_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPnA0_mean",[
                                                              "Prod_name_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnCh0_mean",[
                                                              "Prod_name_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnT0_mean",[
                                                              "Prod_name_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPnSt0_mean",[
                                                              "Prod_name_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SPn0_mean",[
                                                              "Prod_name_ID"],
                                                          ["mean"]),
                                                       ("SBClRACh0_mean",[ ##brand id START
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SBClRA0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SBClRCh0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SBClACh0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SBClR0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SBClA0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SBClCh0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SBCl0_mean",[
                                                              "Brand_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SBR0_mean",[
                                                              "Brand_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SBA0_mean",[
                                                              "Brand_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SBCh0_mean",[
                                                              "Brand_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SBT0_mean",[
                                                              "Brand_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SBSt0_mean",[
                                                              "Brand_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SB0_mean",[
                                                              "Brand_ID"],
                                                          ["mean"])
                                                       
                                                       
                                                       ], "Lag0", True)
print  configLag0Target1DeleteColumnsFalse.__dict__

{'nameAndGroups': [('SPClRACh0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClRA0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID'], ['mean']), ('SPClRCh0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Canal_ID'], ['mean']), ('SPClACh0_mean', ['Producto_ID', 'Cliente_ID', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClR0_mean', ['Producto_ID', 'Cliente_ID', 'Ruta_SAK'], ['mean']), ('SPClA0_mean', ['Producto_ID', 'Cliente_ID', 'Agencia_ID'], ['mean']), ('SPClCh0_mean', ['Producto_ID', 'Cliente_ID', 'Canal_ID'], ['mean']), ('SPCl0_mean', ['Producto_ID', 'Cliente_ID'], ['mean']), ('SPR0_mean', ['Producto_ID', 'Ruta_SAK'], ['mean']), ('SPA0_mean', ['Producto_ID', 'Agencia_ID'], ['mean']), ('SPCh0_mean', ['Producto_ID', 'Canal_ID'], ['mean']), ('SPT0_mean', ['Producto_ID', 'Town_ID'], ['mean']), ('SPSt0_mean', ['Producto_ID', 'State_ID'], ['mean']), ('SP0_mean', ['Producto_ID'], ['mean']), ('SPnClRACh0_mean', ['Prod_name_ID', 'Clien

In [23]:
FE.AddConfigurableFeaturesToTrain(configLag0Target1DeleteColumnsFalse)

SPClRACh0_mean is not in columns..


SPClRACh0_mean
Producto_ID Cliente_ID Ruta_SAK Agencia_ID Canal_ID                
41          146030     3303     2281       7               4.069195
            465617     3306     2281       7               0.000000

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,SPClRACh0_mean
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,2008,16,120,2,709,4,3.0,False,NaN,1.418015
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,2008,16,135,2,712,4,4.0,False,NaN,1.279644


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,SPClRACh0_mean
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23,NaN,1.098612
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23,NaN,NaN


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,SPClRACh0_mean
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,NaN,1.609438
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,NaN,NaN


...

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,SB0_mean
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438,1.666353
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.006477,1.564475


Count of missing numbers after SB0_mean in validation part 1 in column Lag0 is 2
Count of missing numbers after SB0_mean in validation part 2 in column Lag0 is 3


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,1.386294,2008,16,120,2,709,4,3.0,False,1.418015
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,1.609438,2008,16,135,2,712,4,4.0,False,1.279644


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23,1.098612
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23,1.892663


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.006477


In [24]:
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)

In [25]:
FE.SaveDataFrameToHdf('both')

In [26]:
print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag0))
#print ('RMSLE Score:', rmse(FE.test1.Demanda_uni_equil, FE.test1.Lag0))
#print ('RMSLE Score:', rmse(FE.test2.Demanda_uni_equil, FE.test2.Lag0))

('RMSLE Score:', 0.32245654993349293)


In [7]:
configLag0Target1DeleteColumnsFalse = ConfigElements(1,[ ("SPClRACh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClRA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPClRCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClACh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClR0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPClA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPClCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPCl0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPR0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPT0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPSt0_mean",["Semana",
                                                              "Producto_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SP0_mean",["Semana",
                                                              "Producto_ID"],
                                                          ["mean"]),
                                                      ("SPnClRACh0_mean",["Semana",  ##PRODNAMES START
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClRA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPnClRCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClACh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClR0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPnClA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnClCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnCl0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPnR0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPnA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnT0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPnSt0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SPn0_mean",["Semana",
                                                              "Prod_name_ID"],
                                                          ["mean"])

                                                       
                                                       ], "Lag1", True)
print  configLag0Target1DeleteColumnsFalse.__dict__

{'nameAndGroups': [('SPClRACh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClRA0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID'], ['mean']), ('SPClRCh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Canal_ID'], ['mean']), ('SPClACh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClR0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK'], ['mean']), ('SPClA0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Agencia_ID'], ['mean']), ('SPClCh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Canal_ID'], ['mean']), ('SPCl0_mean', ['Semana', 'Producto_ID', 'Cliente_ID'], ['mean']), ('SPR0_mean', ['Semana', 'Producto_ID', 'Ruta_SAK'], ['mean']), ('SPA0_mean', ['Semana', 'Producto_ID', 'Agencia_ID'], ['mean']), ('SPCh0_mean', ['Semana', 'Producto_ID', 'Canal_ID'], ['mean']), ('SPT0_mean', ['Semana', 'Producto_ID', 'Town_ID'], ['mean']), ('SPSt0_mean', ['Se

In [8]:
FE.AddConfigurableFeaturesToTrain(configLag0Target1DeleteColumnsFalse)

SPClRACh0_mean is not in columns..


SPClRACh0_mean
Semana Producto_ID Cliente_ID Ruta_SAK Agencia_ID Canal_ID                
4      41          681747     3306     2281       7               7.632886
                   684023     3303     2281       7               3.433987

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1,SPClRACh0_mean
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,16,120,2,709,4,3.0,False,1.418015,NaN,NaN
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,16,135,2,712,4,4.0,False,1.279644,NaN,NaN


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag1,SPClRACh0_mean
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23,1.098612,NaN,1.098612
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23,1.892663,NaN,NaN


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.006477


...

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.006477


Count of missing numbers after SPn0_mean in validation part 1 in column Lag1 is 1231


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,2008,16,120,2,709,4,3.0,False,1.418015,NaN
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,2008,16,135,2,712,4,4.0,False,1.279644,NaN


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag1
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23,1.098612,1.098612
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23,1.892663,1.853309


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.006477


In [9]:
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)

In [10]:
FE.SaveDataFrameToHdf('both')

In [11]:
FE.test2[(FE.test2.id == 0)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438


In [12]:
configLag0Target1DeleteColumnsFalse = ConfigElements(2,[ ("SPClRACh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClRA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPClRCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClACh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClR0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPClA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPClCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPCl0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPR0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPT0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPSt0_mean",["Semana",
                                                              "Producto_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SP0_mean",["Semana",
                                                              "Producto_ID"],
                                                          ["mean"]),
                                                      ("SPnClRACh0_mean",["Semana",  ##PRODNAMES START
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClRA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPnClRCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClACh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClR0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPnClA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnClCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnCl0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPnR0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPnA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnT0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPnSt0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SPn0_mean",["Semana",
                                                              "Prod_name_ID"],
                                                          ["mean"])

                                                       
                                                       ], "Lag2", True)
print  configLag0Target1DeleteColumnsFalse.__dict__

{'nameAndGroups': [('SPClRACh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClRA0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID'], ['mean']), ('SPClRCh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Canal_ID'], ['mean']), ('SPClACh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClR0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK'], ['mean']), ('SPClA0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Agencia_ID'], ['mean']), ('SPClCh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Canal_ID'], ['mean']), ('SPCl0_mean', ['Semana', 'Producto_ID', 'Cliente_ID'], ['mean']), ('SPR0_mean', ['Semana', 'Producto_ID', 'Ruta_SAK'], ['mean']), ('SPA0_mean', ['Semana', 'Producto_ID', 'Agencia_ID'], ['mean']), ('SPCh0_mean', ['Semana', 'Producto_ID', 'Canal_ID'], ['mean']), ('SPT0_mean', ['Semana', 'Producto_ID', 'Town_ID'], ['mean']), ('SPSt0_mean', ['Se

In [13]:
FE.AddConfigurableFeaturesToTrain(configLag0Target1DeleteColumnsFalse)

SPClRACh0_mean is not in columns..


SPClRACh0_mean
Semana Producto_ID Cliente_ID Ruta_SAK Agencia_ID Canal_ID                
5      41          681747     3306     2281       7               7.632886
                   684023     3303     2281       7               3.433987

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1,Lag2,SPClRACh0_mean
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,120,2,709,4,3.0,False,1.418015,0.0,NaN,NaN
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,135,2,712,4,4.0,False,1.279644,0.0,NaN,NaN


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag1,Lag2,SPClRACh0_mean
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23,1.098612,1.098612,NaN,1.098612
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23,1.892663,1.853309,NaN,NaN


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag2,SPClRACh0_mean
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438,NaN,1.609438
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.006477,NaN,NaN


...

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag2,SPn0_mean
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438,1.609438,2.102888
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.006477,0.975791,1.193880


Count of missing numbers after SPn0_mean in validation part 1 in column Lag2 is 47426
Count of missing numbers after SPn0_mean in validation part 2 in column Lag2 is 17845


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1,Lag2
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,16,120,2,709,4,3.0,False,1.418015,0.0,NaN
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,16,135,2,712,4,4.0,False,1.279644,0.0,NaN


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag1,Lag2
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23,1.098612,1.098612,1.098612
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23,1.892663,1.853309,1.932827


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag2
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438,1.609438
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.006477,0.975791


In [14]:
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)

In [15]:
FE.SaveDataFrameToHdf('both')

In [16]:
FE.test2[(FE.test2.id == 0)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag2
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438,1.609438


In [17]:
configLag0Target1DeleteColumnsFalse = ConfigElements(3,[ ("SPClRACh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClRA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPClRCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClACh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPClR0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPClA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPClCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPCl0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPR0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPA0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPCh0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPT0_mean",["Semana",
                                                              "Producto_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPSt0_mean",["Semana",
                                                              "Producto_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SP0_mean",["Semana",
                                                              "Producto_ID"],
                                                          ["mean"]),
                                                      ("SPnClRACh0_mean",["Semana",  ##PRODNAMES START
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClRA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                        ("SPnClRCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClACh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                        ("SPnClR0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                        ("SPnClA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnClCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnCl0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Cliente_ID"],
                                                          ["mean"]),
                                                       ("SPnR0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Ruta_SAK"],
                                                          ["mean"]),
                                                       ("SPnA0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Agencia_ID"],
                                                          ["mean"]),
                                                       ("SPnCh0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Canal_ID"],
                                                          ["mean"]),
                                                       ("SPnT0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "Town_ID"],
                                                          ["mean"]),
                                                       ("SPnSt0_mean",["Semana",
                                                              "Prod_name_ID",
                                                              "State_ID"],
                                                          ["mean"]),
                                                       ("SPn0_mean",["Semana",
                                                              "Prod_name_ID"],
                                                          ["mean"])

                                                       
                                                       ], "Lag3", True)
print  configLag0Target1DeleteColumnsFalse.__dict__

{'nameAndGroups': [('SPClRACh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClRA0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Agencia_ID'], ['mean']), ('SPClRCh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK', 'Canal_ID'], ['mean']), ('SPClACh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Agencia_ID', 'Canal_ID'], ['mean']), ('SPClR0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Ruta_SAK'], ['mean']), ('SPClA0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Agencia_ID'], ['mean']), ('SPClCh0_mean', ['Semana', 'Producto_ID', 'Cliente_ID', 'Canal_ID'], ['mean']), ('SPCl0_mean', ['Semana', 'Producto_ID', 'Cliente_ID'], ['mean']), ('SPR0_mean', ['Semana', 'Producto_ID', 'Ruta_SAK'], ['mean']), ('SPA0_mean', ['Semana', 'Producto_ID', 'Agencia_ID'], ['mean']), ('SPCh0_mean', ['Semana', 'Producto_ID', 'Canal_ID'], ['mean']), ('SPT0_mean', ['Semana', 'Producto_ID', 'Town_ID'], ['mean']), ('SPSt0_mean', ['Se

In [18]:
FE.AddConfigurableFeaturesToTrain(configLag0Target1DeleteColumnsFalse)

/home/cankoklu/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


SPClRACh0_mean is not in columns..


SPClRACh0_mean
Semana Producto_ID Cliente_ID Ruta_SAK Agencia_ID Canal_ID                
6      41          681747     3306     2281       7               7.632886
                   684023     3303     2281       7               3.433987

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1,Lag2,Lag3,SPClRACh0_mean
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,2,709,4,3.0,False,1.418015,0.0,0.0,NaN,NaN
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,2,712,4,4.0,False,1.279644,0.0,0.0,NaN,NaN


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag1,Lag2,Lag3,SPClRACh0_mean
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23,1.098612,1.098612,1.098612,NaN,NaN
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23,1.892663,1.853309,1.932827,NaN,NaN


...

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag2,Lag3,SPn0_mean
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438,1.609438,1.665441,2.101904
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.006477,0.975791,0.988282,1.193254


Count of missing numbers after SPn0_mean in validation part 1 in column Lag3 is 47712
Count of missing numbers after SPn0_mean in validation part 2 in column Lag3 is 69110


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,weight,pieces,Prod_name_ID,Brand_ID,Demanda_uni_equil_original,DemandaNotEqualTheDifferenceOfVentaUniAndDev,Lag0,Lag1,Lag2,Lag3
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,120,2,709,4,3.0,False,1.418015,0.0,0.0,NaN
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,135,2,712,4,4.0,False,1.279644,0.0,0.0,NaN


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag1,Lag2,Lag3
0,2,10,2045,1,2831,4549769,32940,2163,14,110,10,678,23,1.098612,1.098612,1.098612,1.353182
1,7,10,1612,1,2837,4414012,35305,2352,30,75,0,45,23,1.892663,1.853309,1.932827,1.891364


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag2,Lag3
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438,1.609438,1.665441
1,1,11,2237,1,1226,4705135,1238,2402,4,140,2,630,4,1.006477,0.975791,0.988282


In [19]:
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)

In [20]:
FE.SaveDataFrameToHdf('both')

In [21]:
FE.test2[(FE.test2.id == 0)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag2,Lag3
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438,1.609438,1.665441


In [22]:
print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag0))
#print ('RMSLE Score:', rmse(FE.test1.Demanda_uni_equil, FE.test1.Lag0))
#print ('RMSLE Score:', rmse(FE.test2.Demanda_uni_equil, FE.test2.Lag0))


print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag1))
#print ('RMSLE Score:', rmse(FE.test1.Demanda_uni_equil, FE.test1.Lag1))
#print ('RMSLE Score:', rmse(FE.test2.Demanda_uni_equil, FE.test2.Lag1))


print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag2))
#print ('RMSLE Score:', rmse(FE.test1.Demanda_uni_equil, FE.test1.Lag2))
#print ('RMSLE Score:', rmse(FE.test2.Demanda_uni_equil, FE.test2.Lag2))


print ('RMSLE Score:', rmse(FE.train.Demanda_uni_equil, FE.train.Lag3))
#print ('RMSLE Score:', rmse(FE.test1.Demanda_uni_equil, FE.test1.Lag3))
#print ('RMSLE Score:', rmse(FE.test2.Demanda_uni_equil, FE.test2.Lag3))

('RMSLE Score:', 0.32245654993349293)
('RMSLE Score:', 0.86710360707858425)
('RMSLE Score:', 1.0972761102413859)
('RMSLE Score:', 1.2783695172331342)


In [23]:
FE.test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3538385 entries, 0 to 3538384
Data columns (total 17 columns):
id              uint32
Semana          uint8
Agencia_ID      uint16
Canal_ID        uint8
Ruta_SAK        uint16
Cliente_ID      uint32
Producto_ID     uint16
Town_ID         uint16
State_ID        uint8
weight          uint16
pieces          uint8
Prod_name_ID    uint16
Brand_ID        uint8
Lag0            float64
Lag1            float64
Lag2            float64
Lag3            float64
dtypes: float64(4), uint16(6), uint32(2), uint8(5)
memory usage: 219.3 MB


In [24]:
FE.test2[(FE.test2.id == 0)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,pieces,Prod_name_ID,Brand_ID,Lag0,Lag2,Lag3
0,0,11,4037,1,2209,4639078,35305,2499,28,75,0,45,23,1.609438,1.609438,1.665441


In [25]:
FE.train.loc[:,"weightppieces"] = (FE.train["weight"].values/FE.train["pieces"].values)
FE.test1.loc[:,"weightppieces"] = (FE.test1["weight"].values/FE.test1["pieces"].values)
FE.test2.loc[:,"weightppieces"] = (FE.test2["weight"].values/FE.test2["pieces"].values)
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)
gc.collect()
FE.SaveDataFrameToHdf('both')
client_sum_venta_hoy = FE.train.groupby('Cliente_ID',as_index=False)['Venta_hoy'].sum()
client_sum_venta_uni_hoy = FE.train.groupby('Cliente_ID',as_index=False)['Venta_uni_hoy'].sum()
client_sum = pd.merge(client_sum_venta_hoy, client_sum_venta_uni_hoy, on='Cliente_ID', how='left', sort=False,copy=False)
client_sum.loc[:,"client_sum_venta_div_venta_uni"] = (client_sum["Venta_hoy"].values/client_sum["Venta_uni_hoy"].values)
client_sum.columns = ['Cliente_ID', 'Client_Sum_Venta_hoy', 'Client_Sum_Venta_uni_hoy', 'Client_Sum_venta_div_venta_uni']
client_sum[['Cliente_ID']]=client_sum[['Cliente_ID']].astype('uint32')
client_sum[['Client_Sum_Venta_hoy']]=client_sum[['Client_Sum_Venta_hoy']].astype('float32')
client_sum[['Client_Sum_Venta_uni_hoy']]=client_sum[['Client_Sum_Venta_uni_hoy']].astype('float32')
client_sum[['Client_Sum_venta_div_venta_uni']]=client_sum[['Client_Sum_venta_div_venta_uni']].astype('float32')
FE.train = pd.merge(FE.train, client_sum, on='Cliente_ID', how='left', sort=False, copy=False)
gc.collect()
prod_name_sum_venta_hoy = FE.train.groupby('Prod_name_ID',as_index=False)['Venta_hoy'].sum()
prod_name_sum_venta_uni_hoy = FE.train.groupby('Prod_name_ID',as_index=False)['Venta_uni_hoy'].sum()
prod_name_sum = pd.merge(prod_name_sum_venta_hoy, prod_name_sum_venta_uni_hoy, on='Prod_name_ID', how='left', sort=False,copy=False)
prod_name_sum.loc[:,"prod_name_sum_venta_div_venta_uni"] = (prod_name_sum["Venta_hoy"].values/prod_name_sum["Venta_uni_hoy"].values)
prod_name_sum.columns = ['Prod_name_ID', 'prod_name_sum_Venta_hoy', 'prod_name_sum_Venta_uni_hoy', 'prod_name_sum_venta_div_venta_uni']
prod_name_sum[['Prod_name_ID']]=prod_name_sum[['Prod_name_ID']].astype('uint16')
prod_name_sum[['prod_name_sum_Venta_hoy']]=prod_name_sum[['prod_name_sum_Venta_hoy']].astype('float32')
prod_name_sum[['prod_name_sum_Venta_uni_hoy']]=prod_name_sum[['prod_name_sum_Venta_uni_hoy']].astype('float32')
prod_name_sum[['prod_name_sum_venta_div_venta_uni']]=prod_name_sum[['prod_name_sum_venta_div_venta_uni']].astype('float32')
FE.train = pd.merge(FE.train, prod_name_sum, on='Prod_name_ID', how='left', sort=False, copy=False)
gc.collect()
Producto_sum_venta_hoy = FE.train.groupby('Producto_ID',as_index=False)['Venta_hoy'].sum()
Producto_sum_venta_uni_hoy = FE.train.groupby('Producto_ID',as_index=False)['Venta_uni_hoy'].sum()
Producto_sum = pd.merge(Producto_sum_venta_hoy, Producto_sum_venta_uni_hoy, on='Producto_ID', how='left', sort=False,copy=False)
Producto_sum.loc[:,"Producto_sum_venta_div_venta_uni"] = (Producto_sum["Venta_hoy"].values/Producto_sum["Venta_uni_hoy"].values)
Producto_sum.columns = ['Producto_ID', 'Producto_sum_Venta_hoy', 'Producto_sum_Venta_uni_hoy', 'Producto_sum_venta_div_venta_uni']
Producto_sum[['Producto_ID']]=Producto_sum[['Producto_ID']].astype('uint16')
Producto_sum[['Producto_sum_Venta_hoy']]=Producto_sum[['Producto_sum_Venta_hoy']].astype('float32')
Producto_sum[['Producto_sum_Venta_uni_hoy']]=Producto_sum[['Producto_sum_Venta_uni_hoy']].astype('float32')
Producto_sum[['Producto_sum_venta_div_venta_uni']]=Producto_sum[['Producto_sum_venta_div_venta_uni']].astype('float32')
FE.train = pd.merge(FE.train, Producto_sum, on='Producto_ID', how='left', sort=False, copy=False)
gc.collect()
FE.test1 = pd.merge(FE.test1, client_sum, on='Cliente_ID', how='left', sort=False, copy=False)
FE.test2 = pd.merge(FE.test2, client_sum, on='Cliente_ID', how='left', sort=False, copy=False)
FE.test1 = pd.merge(FE.test1, prod_name_sum, on='Prod_name_ID', how='left', sort=False, copy=False)
FE.test2 = pd.merge(FE.test2, prod_name_sum, on='Prod_name_ID', how='left', sort=False, copy=False)
FE.test1 = pd.merge(FE.test1, Producto_sum, on='Producto_ID', how='left', sort=False, copy=False)
FE.test2 = pd.merge(FE.test2, Producto_sum, on='Producto_ID', how='left', sort=False, copy=False)
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)
gc.collect()
FE.SaveDataFrameToHdf('both')

In [26]:
FE.test2[(FE.test2.id == 0)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,weightppieces,Client_Sum_Venta_hoy,Client_Sum_Venta_uni_hoy,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni
0,0,11,4037,1,2209,4639078,35305,2499,28,75,...,0,311.459991,45.0,6.921333,95029912.0,15018922.0,6.327346,30979202.0,4959501.0,6.246435


In [27]:
FE.SaveDataFrameToHdf('both')

In [28]:
Feature1 = FE.train[["Producto_ID","Demanda_uni_equil","Venta_uni_hoy"]].copy()
gc.collect()
display(Feature1.head(2))
print "Venta_Uni_Hoy = 0: " ,((Feature1.Venta_uni_hoy == 0).sum())
print "Shape of New Dataframe..: ", str((Feature1.shape[0]))
gc.collect()
Feature1 = Feature1[Feature1.Venta_uni_hoy != 0]
print "Shape of New Dataframe after deleting Venta_Uni_Hoy = 0..: ",(Feature1.shape[0])
gc.collect()
Feature1.loc[:,"Demanda_uni_equil"] = np.round( np.expm1(Feature1["Demanda_uni_equil"].values) )
display(Feature1[Feature1.Demanda_uni_equil.values != Feature1.Venta_uni_hoy.values].head(10))
Feature1 = Feature1.groupby("Producto_ID").sum()
gc.collect()
Feature1.loc[:,"Producto_ID_sum_demanda_divide_sum_venta_uni"] = Feature1.Demanda_uni_equil.values / Feature1.Venta_uni_hoy.values
Feature1 = pd.DataFrame(Feature1["Producto_ID_sum_demanda_divide_sum_venta_uni"])
gc.collect()
print Feature1.mean()
display(Feature1.head(10))
FE.train = FE.train.merge( Feature1, left_on="Producto_ID",
                    right_index=True, how='left', sort=False,copy=False)
FE.test1 = FE.test1.merge( Feature1, left_on="Producto_ID",
                    right_index=True, how='left', sort=False,copy=False)
FE.test2 = FE.test2.merge( Feature1, left_on="Producto_ID",
                    right_index=True, how='left', sort=False,copy=False)
display(FE.train.head(2))
display(FE.test1.head(2))
display(FE.test2.head(2))

,Producto_ID,Demanda_uni_equil,Venta_uni_hoy
0,1212,1.386294,3
1,1216,1.609438,4


Venta_Uni_Hoy = 0:  302559
Shape of New Dataframe..:  74180464
Shape of New Dataframe after deleting Venta_Uni_Hoy = 0..:  73877905


,Producto_ID,Demanda_uni_equil,Venta_uni_hoy
117,47611,1.0,2
284,1309,1.0,2
289,30571,6.0,7
293,34053,0.0,3
383,35144,8.0,10
486,1309,6.0,7
594,30571,0.0,3
1048,31717,2.0,5
1262,1125,0.0,16
1275,1250,0.0,2


Producto_ID_sum_demanda_divide_sum_venta_uni    0.958205
dtype: float64


,Producto_ID_sum_demanda_divide_sum_venta_uni
Producto_ID,
41,0.998308
53,1.000000
72,0.990186
73,0.977437
100,0.937393
106,0.983566
107,1.000000
108,1.000000
123,0.995486


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Client_Sum_Venta_hoy,Client_Sum_Venta_uni_hoy,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,6151.200195,886.0,6.942664,67682896.0,5835478.0,11.598517,29280448.0,3499121.0,8.367944,0.981365
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,6151.200195,886.0,6.942664,56129740.0,5398186.0,10.397889,26691390.0,3181308.0,8.390068,0.984187


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,Client_Sum_Venta_hoy,Client_Sum_Venta_uni_hoy,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni
0,2,10,2045,1,2831,4549769,32940,2163,14,110,...,602.799988,171.0,3.525146,9845327.0,1255968.0,7.838836,7877500.5,1009521.0,7.803206,0.994470
1,7,10,1612,1,2837,4414012,35305,2352,30,75,...,191.419998,31.0,6.174839,95029912.0,15018922.0,6.327346,30979202.0,4959501.0,6.246435,0.995613


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,Client_Sum_Venta_hoy,Client_Sum_Venta_uni_hoy,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni
0,0,11,4037,1,2209,4639078,35305,2499,28,75,...,311.459991,45.0,6.921333,95029912.0,15018922.0,6.327346,30979202.0,4959501.0,6.246435,0.995613
1,1,11,2237,1,1226,4705135,1238,2402,4,140,...,824.080017,104.0,7.923846,43785060.0,4384495.0,9.986341,38339704.0,3852209.0,9.952654,0.982354


In [29]:
FE.test2[(FE.test2.id == 0)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,Client_Sum_Venta_hoy,Client_Sum_Venta_uni_hoy,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni
0,0,11,4037,1,2209,4639078,35305,2499,28,75,...,311.459991,45.0,6.921333,95029912.0,15018922.0,6.327346,30979202.0,4959501.0,6.246435,0.995613


In [30]:
Feature1 = FE.train[["Prod_name_ID","Demanda_uni_equil","Venta_uni_hoy"]].copy()
gc.collect()
display(Feature1.head(2))
print "Venta_Uni_Hoy = 0: " ,((Feature1.Venta_uni_hoy == 0).sum())
print "Shape of New Dataframe..: ", str((Feature1.shape[0]))
gc.collect()
Feature1 = Feature1[Feature1.Venta_uni_hoy != 0]
print "Shape of New Dataframe after deleting Venta_Uni_Hoy = 0..: ",(Feature1.shape[0])
gc.collect()
Feature1.loc[:,"Demanda_uni_equil"] = np.round( np.expm1(Feature1["Demanda_uni_equil"].values) )
display(Feature1[Feature1.Demanda_uni_equil.values != Feature1.Venta_uni_hoy.values].head(10))
Feature1 = Feature1.groupby("Prod_name_ID").sum()
gc.collect()
Feature1.loc[:,"Prod_name_ID_sum_demanda_divide_sum_venta_uni"] = Feature1.Demanda_uni_equil.values / Feature1.Venta_uni_hoy.values
Feature1 = pd.DataFrame(Feature1["Prod_name_ID_sum_demanda_divide_sum_venta_uni"])
gc.collect()
print Feature1.mean()
display(Feature1.head(10))
FE.train = FE.train.merge( Feature1, left_on="Prod_name_ID",
                    right_index=True, how='left', sort=False,copy=False)
FE.test1 = FE.test1.merge( Feature1, left_on="Prod_name_ID",
                    right_index=True, how='left', sort=False,copy=False)
FE.test2 = FE.test2.merge( Feature1, left_on="Prod_name_ID",
                    right_index=True, how='left', sort=False,copy=False)
display(FE.train.head(2))
display(FE.test1.head(2))
display(FE.test2.head(2))

,Prod_name_ID,Demanda_uni_equil,Venta_uni_hoy
0,709,1.386294,3
1,712,1.609438,4


Venta_Uni_Hoy = 0:  302559
Shape of New Dataframe..:  74180464
Shape of New Dataframe after deleting Venta_Uni_Hoy = 0..:  73877905


,Prod_name_ID,Demanda_uni_equil,Venta_uni_hoy
117,78,1.0,2
284,82,1.0,2
289,45,6.0,7
293,544,0.0,3
383,961,8.0,10
486,82,6.0,7
594,45,0.0,3
1048,71,2.0,5
1262,876,0.0,16
1275,266,0.0,2


Prod_name_ID_sum_demanda_divide_sum_venta_uni    0.956435
dtype: float64


,Prod_name_ID_sum_demanda_divide_sum_venta_uni
Prod_name_ID,
0,0.984787
1,0.938094
2,0.942642
3,0.974613
4,0.966480
5,0.975662
6,0.955625
9,0.979567
10,0.979104


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Client_Sum_Venta_uni_hoy,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,886.0,6.942664,67682896.0,5835478.0,11.598517,29280448.0,3499121.0,8.367944,0.981365,0.979522
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,886.0,6.942664,56129740.0,5398186.0,10.397889,26691390.0,3181308.0,8.390068,0.984187,0.982811


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,Client_Sum_Venta_uni_hoy,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni
0,2,10,2045,1,2831,4549769,32940,2163,14,110,...,171.0,3.525146,9845327.0,1255968.0,7.838836,7877500.5,1009521.0,7.803206,0.994470,0.994717
1,7,10,1612,1,2837,4414012,35305,2352,30,75,...,31.0,6.174839,95029912.0,15018922.0,6.327346,30979202.0,4959501.0,6.246435,0.995613,0.993341


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,Client_Sum_Venta_uni_hoy,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni
0,0,11,4037,1,2209,4639078,35305,2499,28,75,...,45.0,6.921333,95029912.0,15018922.0,6.327346,30979202.0,4959501.0,6.246435,0.995613,0.993341
1,1,11,2237,1,1226,4705135,1238,2402,4,140,...,104.0,7.923846,43785060.0,4384495.0,9.986341,38339704.0,3852209.0,9.952654,0.982354,0.982280


In [31]:
FE.test2[(FE.test2.id == 0)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,Client_Sum_Venta_uni_hoy,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni
0,0,11,4037,1,2209,4639078,35305,2499,28,75,...,45.0,6.921333,95029912.0,15018922.0,6.327346,30979202.0,4959501.0,6.246435,0.995613,0.993341


In [32]:
Feature1 = FE.train[["Cliente_ID","Demanda_uni_equil","Venta_uni_hoy"]].copy()
gc.collect()
display(Feature1.head(2))
print "Venta_Uni_Hoy = 0: " ,((Feature1.Venta_uni_hoy == 0).sum())
print "Shape of New Dataframe..: ", str((Feature1.shape[0]))
gc.collect()
Feature1 = Feature1[Feature1.Venta_uni_hoy != 0]
print "Shape of New Dataframe after deleting Venta_Uni_Hoy = 0..: ",(Feature1.shape[0])
gc.collect()
Feature1.loc[:,"Demanda_uni_equil"] = np.round( np.expm1(Feature1["Demanda_uni_equil"].values) )
display(Feature1[Feature1.Demanda_uni_equil.values != Feature1.Venta_uni_hoy.values].head(10))
Feature1 = Feature1.groupby("Cliente_ID").sum()
gc.collect()
Feature1.loc[:,"Cliente_ID_sum_demanda_divide_sum_venta_uni"] = Feature1.Demanda_uni_equil.values / Feature1.Venta_uni_hoy.values
Feature1 = pd.DataFrame(Feature1["Cliente_ID_sum_demanda_divide_sum_venta_uni"])
gc.collect()
print Feature1.mean()
display(Feature1.head(10))
FE.train = FE.train.merge( Feature1, left_on="Cliente_ID",
                    right_index=True, how='left', sort=False,copy=False)
FE.test1 = FE.test1.merge( Feature1, left_on="Cliente_ID",
                    right_index=True, how='left', sort=False,copy=False)
FE.test2 = FE.test2.merge( Feature1, left_on="Cliente_ID",
                    right_index=True, how='left', sort=False,copy=False)
display(FE.train.head(2))
display(FE.test1.head(2))
display(FE.test2.head(2))

,Cliente_ID,Demanda_uni_equil,Venta_uni_hoy
0,15766,1.386294,3
1,15766,1.609438,4


Venta_Uni_Hoy = 0:  302559
Shape of New Dataframe..:  74180464
Shape of New Dataframe after deleting Venta_Uni_Hoy = 0..:  73877905


,Cliente_ID,Demanda_uni_equil,Venta_uni_hoy
117,319641,1.0,2
284,1603500,1.0,2
289,1603500,6.0,7
293,1603500,0.0,3
383,2331717,8.0,10
486,4532486,6.0,7
594,319684,0.0,3
1048,1690065,2.0,5
1262,384195,0.0,16
1275,817296,0.0,2


Cliente_ID_sum_demanda_divide_sum_venta_uni    0.984532
dtype: float64


,Cliente_ID_sum_demanda_divide_sum_venta_uni
Cliente_ID,
26,0.931775
60,1.000000
65,0.999884
101,1.000000
105,1.000000
106,1.000000
107,0.943492
215,1.000000
262,1.000000


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,6.942664,67682896.0,5835478.0,11.598517,29280448.0,3499121.0,8.367944,0.981365,0.979522,0.997743
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,6.942664,56129740.0,5398186.0,10.397889,26691390.0,3181308.0,8.390068,0.984187,0.982811,0.997743


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni
0,2,10,2045,1,2831,4549769,32940,2163,14,110,...,3.525146,9845327.0,1255968.0,7.838836,7877500.5,1009521.0,7.803206,0.994470,0.994717,1.0
1,7,10,1612,1,2837,4414012,35305,2352,30,75,...,6.174839,95029912.0,15018922.0,6.327346,30979202.0,4959501.0,6.246435,0.995613,0.993341,1.0


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni
0,0,11,4037,1,2209,4639078,35305,2499,28,75,...,6.921333,95029912.0,15018922.0,6.327346,30979202.0,4959501.0,6.246435,0.995613,0.993341,0.911111
1,1,11,2237,1,1226,4705135,1238,2402,4,140,...,7.923846,43785060.0,4384495.0,9.986341,38339704.0,3852209.0,9.952654,0.982354,0.982280,1.000000


In [33]:
FE.test2[(FE.test2.id == 0)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni
0,0,11,4037,1,2209,4639078,35305,2499,28,75,...,6.921333,95029912.0,15018922.0,6.327346,30979202.0,4959501.0,6.246435,0.995613,0.993341,0.911111


In [34]:
FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)
gc.collect()
FE.SaveDataFrameToHdf('both')

In [5]:
FE.ReadHdf('both')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,6.942664,67682896.0,5835478.0,11.598517,29280448.0,3499121.0,8.367944,0.981365,0.979522,0.997743
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,6.942664,56129740.0,5398186.0,10.397889,26691390.0,3181308.0,8.390068,0.984187,0.982811,0.997743


<class 'pandas.core.frame.DataFrame'>
Int64Index: 74180464 entries, 0 to 74180463
Data columns (total 36 columns):
Semana                                           uint8
Agencia_ID                                       uint16
Canal_ID                                         uint8
Ruta_SAK                                         uint16
Cliente_ID                                       uint32
Producto_ID                                      uint16
Venta_uni_hoy                                    uint16
Venta_hoy                                        float32
Dev_uni_proxima                                  uint32
Dev_proxima                                      float32
Demanda_uni_equil                                float64
Town_ID                                          uint16
State_ID                                         uint8
weight                                           uint16
pieces                                           uint8
Prod_name_ID                                  

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni
0,2,10,2045,1,2831,4549769,32940,2163,14,110,...,3.525146,9845327.0,1255968.0,7.838836,7877500.5,1009521.0,7.803206,0.994470,0.994717,1.0
1,7,10,1612,1,2837,4414012,35305,2352,30,75,...,6.174839,95029912.0,15018922.0,6.327346,30979202.0,4959501.0,6.246435,0.995613,0.993341,1.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3538385 entries, 0 to 3538384
Data columns (total 30 columns):
id                                               uint32
Semana                                           uint8
Agencia_ID                                       uint16
Canal_ID                                         uint8
Ruta_SAK                                         uint16
Cliente_ID                                       uint32
Producto_ID                                      uint16
Town_ID                                          uint16
State_ID                                         uint8
weight                                           uint16
pieces                                           uint8
Prod_name_ID                                     uint16
Brand_ID                                         uint8
Lag0                                             float64
Lag1                                             float64
Lag2                                             f

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni
0,0,11,4037,1,2209,4639078,35305,2499,28,75,...,6.921333,95029912.0,15018922.0,6.327346,30979202.0,4959501.0,6.246435,0.995613,0.993341,0.911111
1,1,11,2237,1,1226,4705135,1238,2402,4,140,...,7.923846,43785060.0,4384495.0,9.986341,38339704.0,3852209.0,9.952654,0.982354,0.982280,1.000000


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3460866 entries, 0 to 3460865
Data columns (total 29 columns):
id                                               uint32
Semana                                           uint8
Agencia_ID                                       uint16
Canal_ID                                         uint8
Ruta_SAK                                         uint16
Cliente_ID                                       uint32
Producto_ID                                      uint16
Town_ID                                          uint16
State_ID                                         uint8
weight                                           uint16
pieces                                           uint8
Prod_name_ID                                     uint16
Brand_ID                                         uint8
Lag0                                             float64
Lag2                                             float64
Lag3                                             f

In [6]:
FE.test2[(FE.test2.id == 0)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,Client_Sum_venta_div_venta_uni,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni
0,0,11,4037,1,2209,4639078,35305,2499,28,75,...,6.921333,95029912.0,15018922.0,6.327346,30979202.0,4959501.0,6.246435,0.995613,0.993341,0.911111


In [7]:
FE.test2.shape

(3460866, 29)

In [8]:
clientPerTown = FE.train[["Town_ID","Cliente_ID","Lag0"]].copy()

clientPerTown = clientPerTown.groupby(["Town_ID","Cliente_ID"])

clientPerTown = clientPerTown.count()
gc.collect()

clientPerTown.reset_index(inplace=True)

clientPerTown = clientPerTown.groupby("Town_ID").count()

clientPerTown = pd.DataFrame( { "ClientPerTown" : clientPerTown.Cliente_ID} )

gc.collect()

FE.train = FE.train.merge( clientPerTown, left_on="Town_ID",
                    right_index=True, how='left', sort=False,copy=False)
FE.test1 = FE.test1.merge( clientPerTown, left_on="Town_ID",
                    right_index=True, how='left', sort=False,copy=False)
FE.test2 = FE.test2.merge( clientPerTown, left_on="Town_ID",
                    right_index=True, how='left', sort=False,copy=False)
display(FE.train.head(2))
display(FE.test1.head(2))
display(FE.test2.head(2))

FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)
gc.collect()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,...,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni,ClientPerTown
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,...,67682896.0,5835478.0,11.598517,29280448.0,3499121.0,8.367944,0.981365,0.979522,0.997743,4408
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,...,56129740.0,5398186.0,10.397889,26691390.0,3181308.0,8.390068,0.984187,0.982811,0.997743,4408


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni,ClientPerTown
0,2,10,2045,1,2831,4549769,32940,2163,14,110,...,9845327.0,1255968.0,7.838836,7877500.5,1009521.0,7.803206,0.994470,0.994717,1.0,4180
1,7,10,1612,1,2837,4414012,35305,2352,30,75,...,95029912.0,15018922.0,6.327346,30979202.0,4959501.0,6.246435,0.995613,0.993341,1.0,4139


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni,ClientPerTown
0,0,11,4037,1,2209,4639078,35305,2499,28,75,...,95029912.0,15018922.0,6.327346,30979202.0,4959501.0,6.246435,0.995613,0.993341,0.911111,5795
1,1,11,2237,1,1226,4705135,1238,2402,4,140,...,43785060.0,4384495.0,9.986341,38339704.0,3852209.0,9.952654,0.982354,0.982280,1.000000,7596


1258

In [9]:
FE.test2[(FE.test2.id == 0)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,prod_name_sum_Venta_hoy,prod_name_sum_Venta_uni_hoy,prod_name_sum_venta_div_venta_uni,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni,ClientPerTown
0,0,11,4037,1,2209,4639078,35305,2499,28,75,...,95029912.0,15018922.0,6.327346,30979202.0,4959501.0,6.246435,0.995613,0.993341,0.911111,5795


In [10]:
FE.test2.shape

(3460866, 30)

In [11]:
client_return_sum = FE.train.groupby('Cliente_ID',as_index=False)['DemandaNotEqualTheDifferenceOfVentaUniAndDev'].sum()
client_return_count = FE.train.groupby('Cliente_ID',as_index=False)['DemandaNotEqualTheDifferenceOfVentaUniAndDev'].count()
client_return = pd.merge(client_return_sum, client_return_count, on='Cliente_ID', how='left', sort=False,copy=False)

client_return.loc[:,"Client_return_rate"] = (client_return["DemandaNotEqualTheDifferenceOfVentaUniAndDev_x"].values/client_return["DemandaNotEqualTheDifferenceOfVentaUniAndDev_y"].values)
client_return.columns = ['Cliente_ID', 'client_return_sum', 'client_return_count', 'Client_return_rate']

FE.train = pd.merge(FE.train, client_return, on='Cliente_ID', how='left', sort=False, copy=False)
FE.test1 = pd.merge(FE.test1, client_return, on='Cliente_ID', how='left', sort=False, copy=False)
FE.test2 = pd.merge(FE.test2, client_return, on='Cliente_ID', how='left', sort=False, copy=False)

FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)
gc.collect()

190

In [12]:
FE.test2[(FE.test2.id == 0)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,Producto_sum_Venta_hoy,Producto_sum_Venta_uni_hoy,Producto_sum_venta_div_venta_uni,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni,ClientPerTown,client_return_sum,client_return_count,Client_return_rate
0,0,11,4037,1,2209,4639078,35305,2499,28,75,...,30979202.0,4959501.0,6.246435,0.995613,0.993341,0.911111,5795,1.0,16.0,0.0625


In [13]:
FE.test2.shape

(3460866, 33)

In [14]:
producto_return_sum = FE.train.groupby('Producto_ID',as_index=False)['DemandaNotEqualTheDifferenceOfVentaUniAndDev'].sum()
producto_return_count = FE.train.groupby('Producto_ID',as_index=False)['DemandaNotEqualTheDifferenceOfVentaUniAndDev'].count()
producto_return = pd.merge(producto_return_sum, producto_return_count, on='Producto_ID', how='left', sort=False,copy=False)

producto_return.loc[:,"producto_return_rate"] = (producto_return["DemandaNotEqualTheDifferenceOfVentaUniAndDev_x"].values/producto_return["DemandaNotEqualTheDifferenceOfVentaUniAndDev_y"].values)
producto_return.columns = ['Producto_ID', 'producto_return_sum', 'producto_return_count', 'producto_return_rate']

FE.train = pd.merge(FE.train, producto_return, on='Producto_ID', how='left', sort=False, copy=False)
FE.test1 = pd.merge(FE.test1, producto_return, on='Producto_ID', how='left', sort=False, copy=False)
FE.test2 = pd.merge(FE.test2, producto_return, on='Producto_ID', how='left', sort=False, copy=False)

FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)
gc.collect()

75

In [15]:
FE.test2[(FE.test2.id == 0)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,Producto_ID_sum_demanda_divide_sum_venta_uni,Prod_name_ID_sum_demanda_divide_sum_venta_uni,Cliente_ID_sum_demanda_divide_sum_venta_uni,ClientPerTown,client_return_sum,client_return_count,Client_return_rate,producto_return_sum,producto_return_count,producto_return_rate
0,0,11,4037,1,2209,4639078,35305,2499,28,75,...,0.995613,0.993341,0.911111,5795,1.0,16.0,0.0625,1709.0,537709.0,0.003178


In [16]:
FE.test2.shape

(3460866, 36)

In [17]:
prod_name_return_sum = FE.train.groupby('Prod_name_ID',as_index=False)['DemandaNotEqualTheDifferenceOfVentaUniAndDev'].sum()
prod_name_return_count = FE.train.groupby('Prod_name_ID',as_index=False)['DemandaNotEqualTheDifferenceOfVentaUniAndDev'].count()
producto_return = pd.merge(prod_name_return_sum, prod_name_return_count, on='Prod_name_ID', how='left', sort=False,copy=False)

producto_return.loc[:,"prod_name_return_rate"] = (producto_return["DemandaNotEqualTheDifferenceOfVentaUniAndDev_x"].values/producto_return["DemandaNotEqualTheDifferenceOfVentaUniAndDev_y"].values)
producto_return.columns = ['Prod_name_ID', 'prod_name_return_sum', 'prod_name_return_count', 'prod_name_return_rate']

FE.train = pd.merge(FE.train, producto_return, on='Prod_name_ID', how='left', sort=False, copy=False)
FE.test1 = pd.merge(FE.test1, producto_return, on='Prod_name_ID', how='left', sort=False, copy=False)
FE.test2 = pd.merge(FE.test2, producto_return, on='Prod_name_ID', how='left', sort=False, copy=False)

FE.train.fillna(value=0, inplace=True)
FE.test1.fillna(value=0, inplace=True)
FE.test2.fillna(value=0, inplace=True)
gc.collect()
#FE.SaveDataFrameToHdf('both')
gc.collect()

0

In [18]:
FE.test2[(FE.test2.id == 0)]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Town_ID,State_ID,weight,...,ClientPerTown,client_return_sum,client_return_count,Client_return_rate,producto_return_sum,producto_return_count,producto_return_rate,prod_name_return_sum,prod_name_return_count,prod_name_return_rate
0,0,11,4037,1,2209,4639078,35305,2499,28,75,...,5795,1.0,16.0,0.0625,1709.0,537709.0,0.003178,6286.0,1455315.0,0.004319


In [19]:
FE.test2.shape

(3460866, 39)

In [20]:
FE.SaveDataFrameToHdf('both')